In [ ]:
import nuscenes
import numpy as np

In [ ]:
nusc = nuscenes.NuScenes(version='v1.0-trainval', dataroot='/data/Datasets/nuScenes', verbose=True)

In [ ]:
sample = nusc.sample[13]

In [ ]:
nusc.render_sample_data(nusc.get('sample_data', sample['data']['LIDAR_TOP'])['token'], nsweeps=5)

In [ ]:
lidar_path, boxes, _ = nusc.get_sample_data(nusc.get('sample_data', sample['data']['LIDAR_TOP'])['token'])

In [ ]:
lidar_path

In [ ]:
points = np.fromfile(lidar_path, dtype=np.float32).reshape([-1, 5])[:, :4].T

In [ ]:
points.shape

In [ ]:
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion

In [ ]:
curr_sd_rec = nusc.get('sample_data', nusc.get('sample', sample['token'])['data']['LIDAR_TOP'])
ego_pose = nusc.get('ego_pose', curr_sd_rec['ego_pose_token'])
calibrated_sensor = nusc.get('calibrated_sensor', curr_sd_rec['calibrated_sensor_token'])

In [ ]:
global_from_car = transform_matrix(
                    ego_pose['translation'],
                    Quaternion(ego_pose['rotation']),
                    inverse=False)
car_from_current = transform_matrix(
                    calibrated_sensor['translation'],
                    Quaternion(calibrated_sensor['rotation']),
                    inverse=False)

tm = np.dot(global_from_car, car_from_current)

In [ ]:
nbr_points = points.shape[1]
points[:3, :] = tm.dot(np.vstack((points[:3, :], np.ones(nbr_points))))[:3, :]
points = points.T
print(points.shape)

In [ ]:
for i in range(points.shape[1]):
    print(points[:, i].min(), points[:, i].max())

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.scatter(points[:, 0],points[:, 1])
plt.show()

In [ ]:
scene = nusc.get('scene', nusc.get('sample', sample['token'])['scene_token'])
scene_token = scene['token']
scene_record = nusc.get('scene', scene_token)
log_record = nusc.get('log', scene_record['log_token'])
map_record = nusc.get('map', log_record['map_token'])
map_mask = map_record['mask']

In [ ]:
cropped = map_mask.mask()[8940:10695, 5599: 6969]

In [ ]:
plt.imshow(cropped)

In [ ]:
ego_pose['translation']

In [ ]:
map_pose = np.concatenate(map_mask.to_pixel_coords(ego_pose['translation'][0], ego_pose['translation'][1]))

In [ ]:
map_pose.shape

In [ ]:
map_pose

In [ ]:
filtered = map_mask.is_on_mask(x=points[:, 0], y=points[:, 1])

In [ ]:
filtered_points = points[filtered]

In [ ]:
filtered_points.shape

In [ ]:
plt.scatter(filtered_points[:, 0],filtered_points[:, 1])
plt.show()

In [ ]:
x_min = map_pose[0] - int(50.4/0.1)
x_max = map_pose[0] + int(50.4/0.1)
y_min = map_pose[1] - int(50.4/0.1)
y_max = map_pose[1] + int(50.4/0.1)
cropped2 = map_mask.mask()[y_min:y_max, x_min:x_max, ]

In [ ]:
plt.imshow(cropped2)